In [1]:
# !ls $FLINK_HOME/lib

In [2]:
from pyflink.table import DataTypes, TableEnvironment, EnvironmentSettings
from pyflink.table.expressions import lit

In [3]:
settings = EnvironmentSettings.new_instance().in_batch_mode().use_blink_planner().build()
t_env = TableEnvironment.create(settings)

In [4]:
content = "line Licensed to the Apache Software Foundation ASF under one " \
              "line or more contributor license agreements See the NOTICE file " \
              "line distributed with this work for additional information " \
              "line regarding copyright ownership The ASF licenses this file " \
              "to you under the Apache License Version the " \
              "License you may not use this file except in compliance " \
              "with the License"

with open('test.txt', 'w') as fopen:
    fopen.write('\n'.join(content.split()))

In [5]:
my_source_ddl = """
    create table mySource (
        word VARCHAR
    ) with (
        'connector' = 'filesystem',
        'format' = 'csv',
        'path' = 'test.txt'
    )
"""

my_sink_ddl = f"""
    create table mySink (
        word VARCHAR,
        `count` BIGINT
    ) with (
        'connector' = 'kafka',
        'topic' = 'mysink',
        'properties.bootstrap.servers' = 'broker:9092',
        'properties.group.id' = 'testGroup',
        'scan.startup.mode' = 'earliest-offset',
        'format' = 'json'
    )
"""

t_env.execute_sql(my_source_ddl)
t_env.execute_sql(my_sink_ddl)

In [6]:
tab = t_env.from_path('mySource')
tab

In [7]:
tab.to_pandas().head()

,word
0,line
1,regarding
2,copyright
3,ownership
4,The


In [8]:
tab.group_by(tab.word) \
   .select(tab.word, lit(1).count) \
   .execute_insert('mySink').wait()

In [11]:
# !pip3 install confluent-kafka

In [12]:
from confluent_kafka import Consumer

In [13]:
conf = {'bootstrap.servers': 'broker:9092', 'group.id': 'test', 'session.timeout.ms': 6000,
            'auto.offset.reset': 'earliest'}
c = Consumer(conf)
c.subscribe(['mysink'])

In [15]:
message = c.poll(0)
message

In [16]:
message.value()

b'{"word":"regarding","count":1}'